In [1]:
import numpy as np
from astropy.io import fits
import matplotlib
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import os
from ProjectF import MLAData,classification, Object,storing,Rebin,StandardRebin
import random

In [2]:
TrainingDir = os.path.normpath("D:\Data")
slash =  os.path.normpath("\\")
TrainingFolder =  os.path.normpath("\Training")

In [3]:
PlateDir ="D:"+slash+"Data"+slash+"Full.txt"
with open(PlateDir) as f:
    Spectra_Files = f.read().splitlines()

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Data\\Full.txt'

In [ ]:
wav_log=[]
Full_table = []
PLATEIDs = []
for spectrum in Spectra_Files:
    plate_ = fits.open( TrainingDir+slash+spectrum ,memmap=True)
    primhdu_ = plate_[0]
    hdu =  plate_[1].data
    Full_table.append(hdu)
    PLATEIDs.append(primhdu_.header['Plate'])
    wav_log.append(primhdu_.header['LogWav'])

From (http://www.sdss.org/dr12/spectro/spectro_basics/) : 'About the Spectra', we  know:


<h5><center>log($\lambda_{i+1}$) - log($\lambda_{i}$)=0.0001.</center></h5>

I will use this, and the first pixels central wavelength, to produce the wavelength array for this plate. This will be done by finding the plate wavelength ratio: 

<h3><center>$\frac{\lambda_{i+1}}{\lambda_{i}}= 10^{0.0001}$.</center></h3>


In [5]:
wav_ratio = 10**0.0001
plate_no = 0
all_wav=[]
while plate_no < len(Full_table): 
    append_count=0
    cent_wav = 10**wav_log[plate_no]
    x = Full_table[plate_no][0][0]
    wavelengths = []
    wavelengths.append(cent_wav)
    current_wav = cent_wav
    while append_count < len(x)-1:
        current_wav = current_wav*wav_ratio
        wavelengths.append(current_wav)
        append_count=append_count+1
    plate_no=plate_no+1
    all_wav.append(wavelengths)
   




In [6]:
len(all_wav)

9

#### We shall use $\lambda$ = 3600 as the starting wavelenth. thus the bin size n, in wavelengths is n $\delta \lambda$  = 0.829n

In [7]:
p = 0
rebin = []
weights = []
Bin_Size=10
while p < len(Full_table):
    current_hdu = Full_table[p]
    wavelenth = all_wav[p]
    r, w = StandardRebin(current_hdu,wavelenth, Bin_Size )    
    rebin.append(r)
    weights.append(w)
    p=p+1

In [8]:
All_z  = []
All_n  = []
All_c  = []
i = 0
while i < len(PLATEIDs):
    plate_r  = []
    plate_n  = []
    plate_c  = []
    plate_hdu  = Full_table[i]
    obj = 0
    while obj <len(plate_hdu):
        currentobj = plate_hdu[obj]
        Currentplate_z = currentobj[4]
        Currentplate_n = currentobj[5]
        Currentplate_c= currentobj[1]
        plate_r.append(Currentplate_z)
        plate_n.append(Currentplate_n)
        plate_c.append(Currentplate_c)
        obj = obj +1
    All_z.append(plate_r)
    All_n.append(plate_n)
    All_c.append(plate_c)
    i = i+1



In [9]:
i=0

while i <len(PLATEIDs):
    C_Plate = PLATEIDs[i]
    plate_z  = All_z[i]
    plate_n  = All_n[i]
    plate_c  = All_c[i]
    Current_wav = wav_log[i]
    obj=0
    col1 = fits.Column(name='Name', format='20A', array=np.array(plate_n))
    col2 = fits.Column(name='Rebin_Flux', format='PD()', array=np.array(rebin[i],dtype=np.object))
    col3 = fits.Column(name='Class', format='I', array=np.array(plate_c))
    col4 = fits.Column(name='Weight', format='PD()', array=np.array(weights[i],dtype=np.object))
    col5 = fits.Column(name='Redshift', format='D', array=np.array(plate_z))    
    cols = fits.ColDefs([col1, col2,col3, col4,col5])
    tbhdu = fits.BinTableHDU.from_columns(cols)
    prihdr = fits.Header()
    prihdr['Plate'] = C_Plate
    prihdr['LogWav'] = Current_wav
    prihdu = fits.PrimaryHDU(header=prihdr)
    file_name = TrainingDir+slash+"rebintrain"+slash+np.str(C_Plate)+'StandardReBin'+np.str(Bin_Size)+'.fits'
    thdulist = fits.HDUList([prihdu, tbhdu])
    thdulist.writeto(file_name)
    print(file_name)
    i=i+1

D:\Data\rebintrain\4791StandardReBin10.fits
D:\Data\rebintrain\6970StandardReBin10.fits
D:\Data\rebintrain\5852StandardReBin10.fits
D:\Data\rebintrain\5484StandardReBin10.fits
D:\Data\rebintrain\6290StandardReBin10.fits
D:\Data\rebintrain\6122StandardReBin10.fits
D:\Data\rebintrain\6424StandardReBin10.fits
D:\Data\rebintrain\6723StandardReBin10.fits
D:\Data\rebintrain\6880StandardReBin10.fits
